In [ ]:
#1. Take all the chunk embeddings from dataset for atleast one document.

!pip install wikipedia
from sentence_transformers import SentenceTransformer
import wikipedia
import numpy as np

# Set Wikipedia language
wikipedia.set_lang("en")

# Load document from Wikipedia
page = wikipedia.page("Machine learning", auto_suggest=False)
doc_text = page.content

# Manual chunking function
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

# Create chunks
chunks = chunk_text(doc_text)

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
embeddings = embedder.encode(chunks, show_progress_bar=True)
embeddings = np.array(embeddings)

# Print results
print("Number of chunks:", len(chunks))
print("Embedding shape:", embeddings.shape)
print("First 10 values of first chunk embedding:")
print(embeddings[0][:10])

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=4868758fc5fc5718fbdddf007085b4e0e8f69d204596529866a14af050e48b9a
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Number of chunks: 133
Embedding shape: (133, 384)
First 10 values of first chunk embedding:
[-0.0395863  -0.02957633  0.07323919 -0.01330787 -0.03926896 -0.01302547
 -0.02885747 -0.06253246 -0.03885961 -0.065111  ]


In [ ]:
#2. Create a new Chroma collection named after the department.
import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection(name="CSE")


In [ ]:
collection = client.get_or_create_collection(name="CSE")

collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    documents=chunks,
    embeddings=embeddings.tolist(),
    metadatas=[
        {"source": "Wikipedia", "document": "Machine Learning", "chunk_id": i}
        for i in range(len(chunks))
    ]
)

print("Data inserted successfully into Chroma collection.")


Data inserted successfully into Chroma collection.


In [ ]:
#3. Insert ids, embeddings, chunk text, metadata
import chromadb

# Initialize Chroma client (use default to avoid conflicts)
client = chromadb.Client()

# Create or load collection (use your department name)
collection = client.get_or_create_collection(name="CSE")

# Insert data into Chroma
collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    documents=chunks,
    embeddings=embeddings.tolist(),
    metadatas=[
        {
            "source": "Wikipedia",
            "document": "Machine Learning",
            "chunk_index": i
        }
        for i in range(len(chunks))
    ]
)

print("Chunks, embeddings, IDs, and metadata inserted successfully.")


Chunks, embeddings, IDs, and metadata inserted successfully.


In [ ]:
#4. Write a query to ask something to db
query = "What is machine learning?"

# Encode query
query_embedding = embedder.encode([query]).tolist()

# Retrieve relevant chunks
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

# Display results
print("Query Results:\n")
for i, doc in enumerate(results["documents"][0]):
    print(f"Result {i+1}:")
    print(doc[:500])
    print("-" * 50)


Query Results:

Result 1:
DA) via unsupervised learning.
From a theoretical viewpoint, probably approximately correct learning provides a mathematical and statistical framework for describing machine learning. Most traditional machine learning and deep learning algorithms can be described as empirical risk minimisation under this framework.


== History ==

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym se
--------------------------------------------------
Result 2:
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many pr

In [ ]:
# week1_embeddings_chroma.py

from sentence_transformers import SentenceTransformer
import wikipedia
import numpy as np
import chromadb

# ----------------------------
# 1. Load dataset (Wikipedia)
# ----------------------------
wikipedia.set_lang("en")
page = wikipedia.page("Machine learning", auto_suggest=False)
doc_text = page.content


# ----------------------------
# 2. Chunk the document
# ----------------------------
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


chunks = chunk_text(doc_text)
print("Number of chunks:", len(chunks))


# ----------------------------
# 3. Generate embeddings
# ----------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks, show_progress_bar=True)
embeddings = np.array(embeddings)

print("Embedding vector shape:", embeddings.shape)
print("First 10 values of first embedding:")
print(embeddings[0][:10])


# ----------------------------
# 4. Create Chroma collection
# ----------------------------
client = chromadb.Client()
collection = client.get_or_create_collection(name="CSE")


# ----------------------------
# 5. Insert IDs, embeddings, text, metadata
# ----------------------------
collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    documents=chunks,
    embeddings=embeddings.tolist(),
    metadatas=[
        {
            "source": "Wikipedia",
            "document": "Machine Learning",
            "chunk_id": i
        }
        for i in range(len(chunks))
    ]
)

print("All chunks, embeddings, and metadata inserted into Chroma DB.")


# ----------------------------
# 6. Query the database
# ----------------------------
query = "What is machine learning?"

query_embedding = embedder.encode([query]).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

print("\nQuery Results:\n")
for i, doc in enumerate(results["documents"][0]):
    print(f"Result {i+1}:")
    print(doc[:500])
    print("-" * 60)


Number of chunks: 133


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Embedding vector shape: (133, 384)
First 10 values of first embedding:
[-0.0395863  -0.02957633  0.07323919 -0.01330787 -0.03926896 -0.01302547
 -0.02885747 -0.06253246 -0.03885961 -0.065111  ]
All chunks, embeddings, and metadata inserted into Chroma DB.

Query Results:

Result 1:
DA) via unsupervised learning.
From a theoretical viewpoint, probably approximately correct learning provides a mathematical and statistical framework for describing machine learning. Most traditional machine learning and deep learning algorithms can be described as empirical risk minimisation under this framework.


== History ==

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym se
------------------------------------------------------------
Result 2:
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can 